In [ ]:
pip install transformers faiss-cpu

In [ ]:
import numpy as np
import faiss
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sentence_transformers import SentenceTransformer

articles = [
    {
        "title": "Tesla's Q3 2024 Earnings: A Surprise Beat",
        "content": "Tesla reported record revenue in Q3 2024, exceeding analysts' expectations due to increased Model 3 and Model Y sales."
    },
    {
        "title": "Analysts Upgrade Tesla Stock Following Strong Performance",
        "content": "Several analysts upgraded their ratings on Tesla stock after the Q3 earnings report, citing strong demand for electric vehicles."
    },
    {
        "title": "Global Supply Chain Recovery Boosts Tesla Production",
        "content": "Improved supply chain conditions allowed Tesla to ramp up production, contributing significantly to its stock price surge."
    },
    {
        "title": "Tesla's Innovative Battery Technology Enhances Range",
        "content": "Recent breakthroughs in battery technology have enabled Tesla to improve the range of its electric vehicles significantly."
    },
    {
        "title": "Market Trends Show Growing Demand for EVs",
        "content": "Overall market trends indicate a surge in consumer interest and demand for electric vehicles across multiple regions."
    }
]

# -----------------------------
# Step 1: Precompute Embeddings & Build a Faiss Index
# -----------------------------
docs = [f"{article['title']}. {article['content']}" for article in articles]

embed_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embed_model.encode(docs)
doc_embeddings = np.array(doc_embeddings, dtype=np.float32)
dimension = doc_embeddings.shape[1]

# Build a Faiss index for efficient similarity search.
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

# -----------------------------
# Step 2: Retrieve Documents via Faiss
# -----------------------------
def retrieve_documents(query, top_k=3):
    query_embedding = embed_model.encode([query])
    query_embedding = np.array(query_embedding, dtype=np.float32)
    distances, indices = index.search(query_embedding, top_k)
    retrieved = [articles[i] for i in indices[0]]
    return retrieved

# -----------------------------
# Step 3: Generate a Response Using Flan-T5
# -----------------------------
def generate_response(query, retrieved_articles):
    model_name = 'google/flan-t5-large'
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    prompt = (
        "--- Retrieved Articles ---\n"
    )
    for article in retrieved_articles:
        prompt += f"Title: {article['title']}\nContent: {article['content']}\n\n"
    prompt += "--- End of Articles ---\n\n"
    prompt += f"Question: {query}\n"
    prompt += "Answer (only use the above articles for your answer, do not add any extra information and just say you don't know if the articles do not have the answer): "

    inputs = tokenizer(prompt, return_tensors='pt', truncation=True)

    outputs = model.generate(
        inputs.input_ids,
        max_length=250,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        do_sample=True
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
query = "What were the key factors contributing to Tesla's strong performance in Q3 2024?"
retrieved = retrieve_documents(query, top_k=3)
response = generate_response(query, retrieved)

print(response)

increased Model 3 and Model Y sales


In [ ]:
query = "Who is the CEO of Tesla?"
retrieved = retrieve_documents(query, top_k=3)
response = generate_response(query, retrieved)

print(response)

No one
